In [ ]:
from dotenv import load_dotenv, find_dotenv
assert load_dotenv(find_dotenv(usecwd=False)), "No .env file found, please create one"

from pathlib import Path

import numpy as np
import pandas as pd
import torch
from skopt.space import Categorical, Integer, Real
import drn

from hyperparameter_tune import hyperparameter_tune
from hyperparameter_tuning_objectives import (
    objective_cann,
    objective_ddr,
    objective_drn,
    objective_mdn,
)

torch.set_num_threads(1)

In [ ]:
accelerator = "gpu" if torch.cuda.is_available() else "cpu"
print(f"Using {accelerator} for training.")

In [ ]:
MODEL_DIR = Path("models/real")
MODEL_DIR.mkdir(parents=True, exist_ok=True)

In [ ]:
hp_opts = {
    "n_calls": 200,
    "n_random_starts": 25,
    "verbose": False,
}

In [ ]:
csv_file_path = "data/freMPL1.csv"
df = pd.read_csv(csv_file_path)
claims = df.loc[df["ClaimAmount"] > 0, :]

In [ ]:
# Scaling
target = claims["ClaimAmount"] / 1000
features = claims.drop("ClaimAmount", axis=1)
features = features.drop(
    ["RecordBeg", "RecordEnd", "ClaimInd", "Garage"], axis=1
)  # Drop garage due to missing values

# Convert "VehAge" categories to numeric
features["VehAge"] = features["VehAge"].map(
    {
        "0": 0,
        "1": 1,
        "2": 2,
        "3": 3,
        "4": 4,
        "5": 5,
        "6-7": 6,
        "8-9": 8,
        "10+": 11,
    }
)
feature_names = features.columns

speed_ranges = [speed for speed in np.unique(features["VehMaxSpeed"])]
speed_series = pd.Series(speed_ranges)
mapping = {speed_range: i + 1 for i, speed_range in enumerate(speed_ranges)}
features["VehMaxSpeed"] = features["VehMaxSpeed"].map(mapping)
features["SocioCateg"] = features["SocioCateg"].str.extract("(\d+)").astype(int)

cat_features = [
    "HasKmLimit",
    "Gender",
    "MariStat",
    "VehUsage",
    "VehBody",
    "VehPrice",
    "VehEngine",
    "VehEnergy",
    "VehClass",
    "SocioCateg",
]

num_features = [feature for feature in features.columns if feature not in cat_features]

In [ ]:
# Split and preprocess the data
(
    x_train,
    x_val,
    x_test,
    y_train,
    y_val,
    y_test,
    x_train_raw,
    x_val_raw,
    x_test_raw,
    num_features,
    cat_features,
    all_categories,
    ct,
) = drn.split_and_preprocess(features, target, num_features, cat_features, seed=0)

# Calculate and print statistics for y_train, y_val, y_test
np.max(y_train), np.median(y_train), np.max(y_val), np.median(y_val), np.max(
    y_test
), np.median(y_test)

In [ ]:
X_train = torch.Tensor(x_train.values)
Y_train = torch.Tensor(y_train.values)
X_val = torch.Tensor(x_val.values)
Y_val = torch.Tensor(y_val.values)

In [ ]:
distribution = "gamma"

In [ ]:
# Generate random seeds for the various training runs
np.random.seed(2026)
seeds = [int(s) for s in np.random.randint(0, 2**32 - 1, size=4)]
seeds

### GLM

In [ ]:
glm = drn.GLM.from_statsmodels(X_train, Y_train, distribution=distribution)
torch.save(glm, MODEL_DIR / "glm.pkl")

### CANN

In [ ]:
name = "cann"
path = MODEL_DIR / f"{name}_hp.pkl"

space_cann_real = [
    Integer(1, 6, name="num_hidden_layers"),
    Categorical([32, 64, 128, 256, 512], name="hidden_size"),
    Real(0.0, 0.5, name="dropout_rate"),
    Real(0.0001, 0.01, name="lr"),
    Categorical([64, 128, 256, 512], name="batch_size"),
]
best_models = hyperparameter_tune(
    objective_cann,
    space_cann_real,
    path,
    seed=seeds[0],
    top_n=1,
    gp_minimize_opts=hp_opts,
    X_train=X_train,
    Y_train=Y_train,
    X_val=X_val,
    Y_val=Y_val,
    accelerator=accelerator,
    glm=glm,
    patience=50,
)

best_model = best_models[0]
torch.save(best_model, MODEL_DIR / f"{name}.pkl")

### MDN

In [ ]:
name = "mdn"
path = MODEL_DIR / f"{name}_hp.pkl"

space_mdn_real = [
    Integer(1, 6, name="num_hidden_layers"),
    Categorical([32, 64, 128, 256, 512], name="hidden_size"),
    Real(0.0, 0.5, name="dropout_rate"),
    Real(0.0001, 0.01, name="lr"),
    Integer(2, 10, name="num_components"),
    Categorical([64, 128, 256, 512], name="batch_size"),
]

best_models = hyperparameter_tune(
    objective_mdn,
    space_mdn_real,
    path,
    seed=seeds[1],
    top_n=1,
    gp_minimize_opts=hp_opts,
    X_train=X_train,
    Y_train=Y_train,
    X_val=X_val,
    Y_val=Y_val,
    accelerator=accelerator,
    distribution="gamma",
    patience=50,
)

best_model = best_models[0]
torch.save(best_model, MODEL_DIR / f"{name}.pkl")

### DDR

In [ ]:
name = "ddr"
path = MODEL_DIR / f"{name}_hp.pkl"

space_ddr_real = [
    Integer(1, 6, name="num_hidden_layers"),
    Categorical([32, 64, 128, 256, 512], name="hidden_size"),
    Real(0.0, 0.5, name="dropout_rate"),
    Real(0.0002, 0.01, name="lr"),
    Categorical([0.05, 0.075, 0.1, 0.125, 0.15], name="proportion"),
    Categorical([64, 128, 256, 512], name="batch_size"),
]

best_models = hyperparameter_tune(
    objective_ddr,
    space_ddr_real,
    path,
    seed=seeds[2],
    top_n=1,
    gp_minimize_opts=hp_opts,
    X_train=X_train,
    Y_train=Y_train,
    X_val=X_val,
    Y_val=Y_val,
    accelerator=accelerator,
    patience=50,
)

best_model = best_models[0]
torch.save(best_model, MODEL_DIR / f"{name}.pkl")

### DRN

In [ ]:
name = "drn"
path = MODEL_DIR / f"{name}_hp.pkl"

space_drn_real = [
    Integer(1, 6, name="num_hidden_layers"),
    Categorical([32, 64, 128, 256, 512], name="hidden_size"),
    Real(0.0, 0.5, name="dropout_rate"),
    Real(0.0002, 0.01, name="lr", prior="log-uniform"),
    Real(1e-6, 1e-1, name="kl_alpha", prior="log-uniform"),
    Categorical([0, 1e-6, 1e-5, 1e-4, 1e-3, 1e-2, 1e-1], name="mean_alpha"),
    Categorical([0, 1e-6, 1e-5, 1e-4, 1e-3, 1e-2, 1e-1], name="dv_alpha"),
    Categorical([64, 128, 256, 512], name="batch_size"),
    Categorical([0.1, 0.125, 0.15], name="proportion"),
    Categorical([1, 3, 5], name="min_obs"),
]

best_models = hyperparameter_tune(
    objective_drn,
    space_drn_real,
    path,
    seed=seeds[3],
    top_n=1,
    gp_minimize_opts=hp_opts,
    X_train=X_train,
    Y_train=Y_train,
    X_val=X_val,
    Y_val=Y_val,
    accelerator=accelerator,
    glm=glm,
    criteria="CRPS",
    kl_direction="forwards",
    patience=50,
)

best_model = best_models[0]
torch.save(best_model, MODEL_DIR / f"{name}.pkl")